In [38]:
import json
import requests
import datetime as dt
import pickle
import pandas as pd
import mysql.connector
import os
from os.path import join, dirname
from datetime import date, datetime

from dotenv import load_dotenv
import db_helper

In [39]:
def get_odds_data():
    api_key = 'da3195c8356b3b0ee3e5a6682ed9c537'
    sports_response = requests.get('https://api.the-odds-api.com/v3/odds', params={
        'api_key': api_key,
        'sport': 'basketball_ncaab',
        'region': 'us',
        'oddsFormat': 'american'
    })


#     print(sports_response.headers['x-requests-remaining'])
    sports_json = json.loads(sports_response.text)
    return sports_json
    

In [40]:
print("https://api.the-odds-api.com/v3/odds/?apiKey=da3195c8356b3b0ee3e5a6682ed9c537&sport=basketball_ncaab&region=us&mkt=h2h&oddsFormat=american")

https://api.the-odds-api.com/v3/odds/?apiKey=da3195c8356b3b0ee3e5a6682ed9c537&sport=basketball_ncaab&region=us&mkt=h2h&oddsFormat=american


In [41]:
def getTeamData(teamid):
    teamid = teamid.replace(" ", "%20")
    baseUrl="https://fastbreakstats-server-main-hqc7w3njda-uc.a.run.app/teams/"
    url=baseUrl+teamid

    r = requests.get(url = url).json()
    bpival = r['bpi']
    kpval = r['kp']
    srsval = r['srs']
    return kpval, bpival, srsval

In [42]:
def get_needed_odds(conf):
    #This function calculates the odds needed for a bet to have an expected value of .1
    #An EV of .1 means a $.10 expected return on $1 over time
    #The normal formula is EV = (payout * confidence) - (bet * (1-conf))
    #payout is the expected profit on a bet.
    #confidence is models % chance of winning. 1-conf is for the other team
    #The bet is always going to be $1 for simple math which removes the need for the bet variable
    #below equation is rearranged to isolate and solve for the payout
    payout = round(((.1+(1-conf))/conf),4) 
    #convert payout to american odds
    if 100/payout > 100:
        odds = round(-100/payout)
    else:
        odds = round(100*payout)
    return odds

In [43]:
def get_ev(odds,conf):
    #This calculates the Expected Return on a $1 bet with the actual given bookie odds we pass in
    #EV of .1 means a 10 cent return on a $1 bet over time
    if int(odds) > 0:
        payout = odds/100
    else:
        payout = -100/odds
    ev = (conf * payout) - (1-conf)
    return ev

In [44]:
def get_kelly_bet(odds,conf):
    if int(odds) > 0:
        dec = 1+(odds/100)
    else:
        dec = 1-(100/odds)
    kb = ((dec*conf) - (1-conf))/dec
    return kb

In [45]:
def convert_epoch(epoch):
    return dt.datetime.fromtimestamp(epoch).strftime('%Y-%m-%d %H:%M:00')

In [50]:
def get_best_bets(teams,loaded_model,sports_json):
    games_to_bet = []
    for game in sports_json['data']:
        #Loop through and get odds
        if game['commence_time'] < dt.datetime.now().timestamp():
            continue
        if dt.datetime.fromtimestamp(game['commence_time']).date() > dt.datetime.now().date():
            continue

        game_time = convert_epoch(game['commence_time'])
        home = game['home_team']
        if game['home_team'] == game['teams'][0]:
            away = game['teams'][1]
        else:
            away = game['teams'][0]

        try:
            home = home.replace("-", " ")
            home = home.replace("(", "")
            home = home.replace(")", "")
            home = home.replace(".", "")
            away = away.replace("-", " ")
            away = away.replace("(", "")
            away = away.replace(")", "")
            away = away.replace(".", "")
            homeid = str(teams.loc[teams['odds_api_name'] == home]['teamID'].values[0])
            awayid = str(teams.loc[teams['odds_api_name'] == away]['teamID'].values[0])
            homekp, homebpi, homesrs = getTeamData(homeid)
            awaykp, awaybpi, awaysrs = getTeamData(awayid)
        except Exception as e:
            print(e)
            print(game['teams'])
            continue

        res = loaded_model.predict_proba([[float(awaykp), float(awaybpi), float(awaysrs), float(homekp), float(homebpi), float(homesrs)]])
        away_needed_odds = int(get_needed_odds(res[0][0]))    
        home_needed_odds = int(get_needed_odds(res[0][1]))

    #     print()
        sites = list(filter(lambda site: site['site_key'] in ['williamhill_us','draftkings','bovada'], game['sites']))
        if not sites:
    #         print("Empty")
            continue
    #     print(game['teams'],game['home_team'])
        site_list, update_time_list, home_odds, away_odds = [],[],[],[]
        for site in sites:
            site_list.append(site['site_key'])
            update_time_list.append(convert_epoch(site['last_update']))
            if home == game['teams'][0]:
                home_odds.append(int(site['odds']['h2h'][0]))
                away_odds.append(int(site['odds']['h2h'][1]))
            else:
                home_odds.append(int(site['odds']['h2h'][1]))
                away_odds.append(int(site['odds']['h2h'][0]))


    #     print("Game Time:", game_time)
    #     print('Home team:', homeid)
    #     print('Away team:', awayid)
#         print('Home needed odds:', home_needed_odds)
#         print('Away needed odds:', away_needed_odds)
    #     print('Sites to check:', site_list)
    #     print('Last Updated:', update_time_list)
    #     print('Home Odds', home_odds)
    #     print('Away Odds', away_odds)
    #     print()

        if any(home_odd > home_needed_odds for home_odd in home_odds):
            index = home_odds.index(max(home_odds))
    #         print('Sites to check:', site_list[index])
    #         print('Last Updated:', update_time_list[index])
    #         print('Betting Odds', home_odds[index])
    #         print('Team to Bet ID', homeid)
            game_to_bet = []
            game_to_bet.append(game_time)
            game_to_bet.append(str(homeid))
            game_to_bet.append(str(awayid))
            game_to_bet.append(site_list[index])
            game_to_bet.append(str(home_odds[index]))
            game_to_bet.append(update_time_list[index])
            game_to_bet.append(str(home_needed_odds))
            game_to_bet.append(str(homeid))
            games_to_bet.append(game_to_bet)
        if any(away_odd > away_needed_odds for away_odd in away_odds):
            index = away_odds.index(max(away_odds))
    #         print('Best Site for bet:', site_list[index])
    #         print('Last Updated:', update_time_list[index])
    #         print('Odds', away_odds[index])
    #         print('Team to Bet ID', awayid)
            game_to_bet = []
            game_to_bet.append(game_time)
            game_to_bet.append(str(homeid))
            game_to_bet.append(str(awayid))
            game_to_bet.append(site_list[index])
            game_to_bet.append(str(away_odds[index]))
            game_to_bet.append(update_time_list[index])
            game_to_bet.append(str(away_needed_odds))
            game_to_bet.append(str(awayid))
            games_to_bet.append(game_to_bet)
    return games_to_bet

c:\users\blake\appdata\local\programs\python\python36\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [52]:
url="https://fastbreakstats-server-main-hqc7w3njda-uc.a.run.app/teams"
r = requests.get(url = url).json()
teams = pd.DataFrame(r)
loaded_model = pickle.load(open('6yr_LR.sav', 'rb'))
# games_to_bet = []
sports_json = get_odds_data()

In [55]:
best_bets = get_best_bets(teams,loaded_model,sports_json)

index 0 is out of bounds for axis 0 with size 0
["Hawai'i Rainbow Warriors", 'UC Riverside Highlanders']


In [56]:
for bet in best_bets:
    print(bet)

['2021-01-08 15:00:00', '240', '158', 'williamhill_us', '-145', '2021-01-08 10:22:00', '-279', '240']
['2021-01-08 15:00:00', '198', '245', 'bovada', '-290', '2021-01-08 10:22:00', '-325', '198']
['2021-01-08 16:00:00', '107', '218', 'draftkings', '112', '2021-01-08 10:22:00', '-136', '107']
['2021-01-08 17:00:00', '10', '101', 'williamhill_us', '-140', '2021-01-08 10:22:00', '-193', '10']
['2021-01-08 17:30:00', '301', '219', 'williamhill_us', '-170', '2021-01-08 10:22:00', '-252', '301']
['2021-01-08 18:00:00', '223', '90', 'draftkings', '-360', '2021-01-08 10:22:00', '-374', '223']
['2021-01-08 18:00:00', '172', '238', 'williamhill_us', '190', '2021-01-08 10:22:00', '161', '238']
['2021-01-08 18:00:00', '54', '214', 'williamhill_us', '-130', '2021-01-08 10:22:00', '-151', '54']
['2021-01-08 18:00:00', '201', '284', 'williamhill_us', '130', '2021-01-08 10:22:00', '-110', '284']
['2021-01-08 19:00:00', '39', '148', 'draftkings', '104', '2021-01-08 10:22:00', '-112', '148']
['2021-01-0

In [49]:
db = db_helper.Db_Helper()
db.insert_odds_historical(best_bets)
db.close()